<a href="https://colab.research.google.com/github/CE339/CES-S24/blob/main/Module06/shortpath_toWorkOn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initializing Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

**Installing Pyomo and a solver**

In [ ]:
!pip install pyomo
!apt-get install -y -qq coinor-cbc  #!apt-get install -y -qq glpk-utils

**Importing Pyomo and CBC Solver**

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('cbc',executable='/usr/bin/cbc')

**Building the abstract model**

In [ ]:
from math import inf

model = AbstractModel()

model.N = Set() #Nodes set
model.A = Set(within=model.N*model.N) #Arcs set

#set cost parameters
model.c = Param(model.A, within=PositiveReals)

#decision variables
# Flow in the arc defined as decision variable xij
model.x = Var(model.A, within=NonNegativeReals)

#Objective Function
#minimize the flow cost in the network
def cost_rule(model):
	return sum(model.c[i,j]*model.x[i,j] for (i,j) in model.A)
model.total = Objective(rule=cost_rule, sense=minimize)

#This is the flow balance constraint (flow_rule below)
#for the min cost network flow problem
#some modifications are necessary in order to represent
#the shortest path problem
#think about them and implement!!!
def flow_rule(model, k):
    inFlow  = sum(model.x[i,j] for (i,j) in model.A if j == k)
    outFlow = sum(model.x[i,j] for (i,j) in model.A if i == k)
    return inFlow + model.supply[k] == outFlow
model.flow = Constraint(model.N, rule=flow_rule)

#------------------------------------------------------------------------------
#Suffix for dual variables reporting. See more details at https://pyomo.readthedocs.io/en/stable/working_models.html
#------------------------------------------------------------------------------
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
model.rc = Suffix(direction=Suffix.IMPORT_EXPORT)

**Creating the model instance** - using data contained in the data file shortpath.dat

In [ ]:
instance = model.create_instance('shortpath.dat')
instance.pprint()

**Optimizing the model instance**

In [ ]:
results = opt.solve(instance, tee=True)

**Display Results**

In [ ]:
print(instance.display())

**Looking at the Dual Variables Associated with the Constraints**

In [ ]:
instance.dual.pprint() #Printing Dual Vars
instance.rc.pprint() #Printing Reduced Costs